# Finding Fourier Series Coefficients

This notebook replicates and extends the introduction to Fourier Series in the following text: 

[Practical Signal Processing by Mark Owen](https://books.google.com/books/about/Practical_Signal_Processing.html?id=lx-tqq-MkK0C)

In [2]:
# Imports

import time

from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import gridplot, column
from bokeh.models import Title
from bokeh.plotting import figure 

import numpy as np
from numpy import exp

output_notebook()

Loading BokehJS ...

--------------

# Visual Fourier Explanation

--------------


The Fourier transform and and its coefficients are integral to understanding signal processing, but can be hard to grasp initially.  Mark Owen did a fantastic job of explaining this in an intuitive light in the above book, and so I thought to explore his approach a bit here.  Most of these explanations draw directly from the above.

Imagine taking snapshots (or samples) of motion in a circle.  This motion will have some period and frequency.  If we stack the snapshots in order of time, we see a spiral of motion through the time axis.  

![image](./assets/fourier_raw_signal.JPG)

To determine the frequency of motion, Mark suggests rotating the points backwards by tuning the frequency of rotation.  We rotate the points by multiplying each point by 
$$e^{i\theta}$$
, where $\theta$ is defined as 

$$\tau f $$ 
and f is the rotational frequency.  We vary the frequency and multiply the points by $e^{i\theta}$ until the movie shows a straight line.  The frequency where we get a straight line (or point in 2-d) is the Fourier coefficient for the motion.

![image](./assets/fourier_rotated_signal.JPG)

## Constants

In [3]:
# Number of samples to take of signal
N = 16

# tau >> pi
tau = 2 * np.pi  

# Take one revolution and split into N samples
theta = tau * np.linspace(0, 1, N + 1)[:-1]

## Helpers

In [4]:
def make_points(f, r, theta):
    '''
    Makes points around a circle of radius r, rotating at frequency f
    We choose integer values of f only, to keep the rotations complete 
    and reducing frequency searches to integers only
    '''
    assert type(f) == int
    return r * exp(1j * f * theta)


def rotate_points(points, f, N):
    '''
    Rotate each point by -tau * f + the rotation of the last point
    '''
    points_rot = []
    for k, point in enumerate(points):
        theta = - tau * f * k / N 
        points_rot.append(point * exp(1j * theta))
    # rounding is for plotting
    return np.round(np.array(points_rot), 6)


def plot_points(points):
    '''
    Show (complex) points
    '''
    wid = 250
    p = figure(width=wid, height=wid)
    x, y = points.real, points.imag
    r = p.circle(x, y) 
    show(p)

Note we are working in the complex plane to make the math simpler, so in the above points, the x value is the real portion of each point and the y value is the complex one.

Lets take an example of frequency 3, radius 1 to plot points around a circle, and then see if we can find the frequency using the approach outlined above.

In [5]:
f = 3
r = 1
points = make_points(f, r, theta)

plot_points(points)

Now lets rotate each of the frames of the points by $-\theta$ * the index of the point over total points

In [6]:
plots = []
wid = 250
ran = (-1.5, 1.5)

for frequency in range(1, 5):
    
    points_rot = rotate_points(points, frequency, N)
    center = points_rot.real.mean(), points_rot.imag.mean()

    plot = figure(
        plot_width=wid, plot_height=wid, 
        x_range=(-1.5, 1.5),
        title=f'Rotation frequency: {frequency}'
    )
    plot.circle(points_rot.real, points_rot.imag)
    plot.add_layout(Title(text=f'Centroid: {center}', align="center"), "left")
    plots.append(plot)
    
grid = gridplot([plots[:2], plots[2:]])
show(grid)

Notice that for us, the original signal frequency was 3, and when rotating each point back by 3, the the plot of points converges to one (non-zero) value.  If we take the average of all the points (the centroid), we see that all other plots (having frequency != 3) average to zero, except the one with the correct frequency.  These centroids (scaled by N points) are the fourier coefficients of this signal!

----------

# Epicycles

-----------

[Epicycles](https://en.wikipedia.org/wiki/Deferent_and_epicycle), or 'circles on circles' are a historic method in which all astronomical motions were described.  Any periodic motion can be described by epicycles, but the complexity grows pretty quickly, which is partially why astronomy moved on.  They are, however, a great model for studying Fourier Coefficients.  

Imagine the Moon circling the earch, and the Earth in turn circling the Sun.  The moon's orbit around the sun is an epicycle, or small terrestrial orbits around a much larger solar orbit.  Ignoring precessions for the moment, lets imagine this fully explains the moon's orbital pattern within the solar system.  If we look at the Moon's orbit in terms of frequencies (in Fourier space), we will see a frequency component for its orbit around the Earth, and another for the Earth's orbit around the sun.


In [7]:
# Make N larger so we can see the orbits more clearly
# Number of samples to take of signal
N = 500
theta = tau * np.linspace(0, 1, N + 1)[:-1]

# points of earth's rotation around the sun
earth_freq = 1
points_earth = make_points(earth_freq, 1, theta)

# points of moon's rotation around the earth
moon_freq = 7
points_moon = make_points(moon_freq, .4, theta)

# moon around sun is simply the sum of the above
points = points_earth + points_moon

plot_points(points)

Great!  Now we can see a more complex motion.  We could continue to add orbits, like the precession of the sun and moon about their axix of rotation etc.  In any case, we should see two Fourier components to this motion.  Lets perform the similar rotation of frames for each point and look for the frequencies where the centroid of the points is non-zero.  These will be the Fourier coefficients of this complex motion, namely `earth_freq` and `moon_freq`

In [10]:
def combine_points(f, r, N):
    '''
    turn frequencies and radii into summed list of points
    '''
    theta = tau * np.linspace(0, 1, N + 1)[:-1]

    points = None
    for i in range(len(f)):
        if points is not None:
            points += make_points(f[i], r[i], theta)
        else:
            points = make_points(f[i], r[i], theta)
    return points


def animate_points_ft(frequencies, radii, N=500, max_f=20):
    '''
    plot animation of points, their rotations, and the fourier coefficients
    inputs:
    
        frequencies (list) frequencies of rotations of all the sub-orbits
        radii (list) of radii of all the sub-orbits
        N (int) number of points in a rotation
        max_f (int) max frequency to check fourier coefficients for
        
    !! len(frequencies) needs to be same as len(radii)
    
    '''
    print(f'Frequencies: {frequencies}\n Radii: {radii}\n N: {N} \n Max F: {max_f}')
    points = combine_points(frequencies, radii, N)
    x, y = points.real, points.imag
    
    w = 400
    h = 150

    # Top image:
    p1 = figure(width=w, height=w)
    p1.line(x=(0, 0), y=(-2, 2))
    p1.line(x=(-2, 2), y=(0, 0))
    r = p1.circle(x, y) 
    rm = p1.circle([x.mean()], [y.mean()], color='red', size=8)

    # Bottom image:
    p2 = figure(width=w, height=h, x_range=(-1, max_f + 1), y_range=(0, 2))
    freqs = p2.circle([0], [int(x.mean())], color='red', size=8)

    # get and explicit handle to update the next show cell with
    target = show(column(p1, p2), notebook_handle=True)

    # plot animation of each frequency along with the fourier components
    f = 0
    while True:
        f += 1
        if f == max_f:
            break
        p1.title.text = f'Frequency: {f}'

        # get rotated coordinates
        points_rot = rotate_points(points, f, N)
        x = points_rot.real
        y = points_rot.imag

        # over-write rotated points in top plot
        r.data_source.data['x'] = x
        r.data_source.data['y'] = y
        rm.data_source.data['x'] = [round(x.mean(), 3)]
        rm.data_source.data['y'] = [round(y.mean(), 3)]

        # append current frequency to freqs
        x_freqs = freqs.data_source.data['x'] + [f]
        y_freqs = freqs.data_source.data['y'] + [round(x.mean(), 3)]
        freqs.data_source.data = {'x': x_freqs, 'y': y_freqs}

        # push updates to the plot using the handle
        push_notebook(handle=target)
        time.sleep(.2)

frequencies = [1, 7]
radii = [1, .4]
animate_points_ft(frequencies, radii)

Frequencies: [1, 7]
 Radii: [1, 0.4]
 N: 500 
 Max F: 20


Neat!  We see the non-zero points on the top curve corresponding to the fourier coefficients in the bottom one.  Since the earth going around the sun is a much larger motion, its magnitude (or power) is larger than the moon's about the earth.  The curves of points post rotating I find amazing as well.

## Nyquist Frequency and Aliasing

We can also notice aliasing if we adjust the number of samples `N` and increase the `max_f` frequency at which we search.  Lets try setting `N` to something smaller and scan frequencies above `N`.  We will see repetitions in the fourier components.  This shows another underlying feature of fourier transforms: discrete in one space (here, time) means periodic in the other domain (here, frequency).  In truth, we are discrete and periodic in both spaces in this example, but the aliasing (periodicity) of frequencies here is coming from the discreteness of our time samples.

In [9]:
frequencies = [1, 7]
radii = [1, .4]
animate_points_ft(frequencies, radii, N=20, max_f=60)

Frequencies: [1, 7]
 Radii: [1, 0.4]
 N: 20 
 Max F: 60
